Подключаем библиотеки.


Библиотека string понадобится для обработки строк, а именно для очистки строк от различных знаков препинания.


Библиотека pymorphy2 понадобится для приведения слова к начальной форме.


Библиотека nltk понадобится для загрузки стоп-слов (например, "это", "я", "ты")

In [77]:
import numpy as np
import string
import pymorphy2
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alexe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Предварительно подготавливаем два текстовых файла. В первый из них записываем спам-сообщения, во второй - примеры информативных сообщений.
Построчно читаем оба файла и создаем два соответствующих массива, элементам которых соответствуют прочитанные строки из файла.

In [78]:
with open('spam.txt') as file:
    spam = [row.strip() for row in file]
    
with open('inf.txt') as file:
    inf = [row.strip() for row in file]

Создаем метод по предобработке предложений:

1)переводим буквы из верхнего регистра в нижний


2)убираем из предложений знаки пунктуации


3)убираем из предложения стоп-слова

Затем, с помощью библиотеки pymorphy2 приводим все слова к начальной форме.

In [79]:
def preprocess(name):
    #print(name)
    morph = pymorphy2.MorphAnalyzer(lang='ru')
    result = []

    for line in name:
        
        line = line.lower()
        line = ''.join([t for t in line if t not in string.punctuation])
        line = [t for t in line.split() if t not in stopwords.words('russian')]
        temp = []
        for word in line:
            temp.append(morph.parse(word)[0].normal_form)
        result += [temp]

    return result

Каждому из предложений выборки ставим в соответствие метку 0 или 1 (0 = спам, 1 = не спам)

In [80]:
dataset = []

for item in preprocess(spam):
    dataset += [[item, 0]]
for item in preprocess(inf):
    dataset += [[item, 1]]
    
print(dataset)

[[['привет', 'давать', 'заходить', 'мы', 'новый', 'коллекция', 'платье'], 0], [['сегодня', 'зайти', 'наш', 'сайт', 'получите…'], 0], [['сегодня', 'распродажа'], 0], [['течение', 'неделя', 'смочь', 'купить', 'платье', 'превлекательный', 'цена'], 0], [['сегодня', 'самый', 'день', 'смочь', 'зайти', 'наш', 'магазин', 'воспользоваться', 'большой', 'скидка', 'который', 'давать', 'именно'], 0], [['пожалуйста', 'срочно', 'открыть', 'прочитать', 'это', 'письмо', 'конец'], 0], [['сегодня', 'скидка', '50'], 0], [['хотеть', 'получить', 'наличный', 'деньга', 'бесплатно', 'рассказать', 'получить', 'дополнительный', 'доход', 'благодаря', 'бизнес', 'дом'], 0], [['проверить', 'свой', 'vip', 'статус', 'помощь', 'специальный', 'сервис'], 0], [['зарегистрироваться', 'ссылка', 'ниже', 'добавить', 'друг'], 0], [['купить', 'электронный', 'книга', '«целевой', 'маркетинг»', 'сегодня', '23'], 0], [['заработать', '50000', 'свой', 'следующий', 'сделка'], 0], [['провести', 'опрос', 'среди', 'наш', 'подписчик', 'ша

In [81]:
def fit(dataset, alpha):
    classes, freq, total = {}, {}, set()
    for features, label in dataset:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
        for feature in features:
            if (feature, label) not in freq:
                freq[(feature, label)] = 0
            freq[(feature, label)] += 1
        total.add(tuple(features))
            
    for feature, label in freq:
        freq[(feature, label)] = (alpha + freq[(feature, label)]) / \
        (alpha*len(total) + classes[label])
    for cl in classes:
        classes[cl] /= len(dataset)        
    return classes, freq

def classify(classifier, features):
    classes, freq = classifier
    return max(classes.keys(),\
              key = lambda cl: np.log10(classes[cl]) +\
               sum(np.log10(freq.get((feature, cl), 10**(-7)))\
                   for feature in features))



Сохраняем созданный словарь в виде файла pkl:

In [82]:
classifier = fit(dataset, .5)

In [83]:
import joblib
joblib.dump(classifier, 'dictionary.pkl')

['dictionary.pkl']

In [84]:
import joblib
classr = joblib.load('dictionary.pkl')
print(classr)

({0: 0.5, 1: 0.5}, {('привет', 0): 0.022388059701492536, ('давать', 0): 0.03731343283582089, ('заходить', 0): 0.022388059701492536, ('мы', 0): 0.03731343283582089, ('новый', 0): 0.05223880597014925, ('коллекция', 0): 0.022388059701492536, ('платье', 0): 0.03731343283582089, ('сегодня', 0): 0.11194029850746269, ('зайти', 0): 0.03731343283582089, ('наш', 0): 0.09701492537313433, ('сайт', 0): 0.022388059701492536, ('получите…', 0): 0.022388059701492536, ('распродажа', 0): 0.022388059701492536, ('течение', 0): 0.022388059701492536, ('неделя', 0): 0.022388059701492536, ('смочь', 0): 0.03731343283582089, ('купить', 0): 0.03731343283582089, ('превлекательный', 0): 0.022388059701492536, ('цена', 0): 0.03731343283582089, ('самый', 0): 0.022388059701492536, ('день', 0): 0.022388059701492536, ('магазин', 0): 0.022388059701492536, ('воспользоваться', 0): 0.022388059701492536, ('большой', 0): 0.022388059701492536, ('скидка', 0): 0.05223880597014925, ('который', 0): 0.03731343283582089, ('именно', 0

Предобрабатываем тестовое предложение и запускаем классификатор для него:

In [85]:
test1 = ['На следующей неделе консультация со студ. активом?']

print(preprocess(test1)[0])
classify(classr, preprocess(test1)[0])

['следующий', 'неделя', 'консультация', 'студа', 'актив']


1

In [86]:
test2 = ['Сегодня распродажа, скидки 50% на все, только VIP пользователям!']

print(preprocess(test2)[0])
classify(classr, preprocess(test2)[0])

['сегодня', 'распродажа', 'скидка', '50', 'vip', 'пользователь']


0

In [87]:
test3 = ['Зарегистрируйся по ссылке ниже и добавь меня в друзья.']

print(preprocess(test3)[0])
classify(classr, preprocess(test3)[0])

['зарегистрироваться', 'ссылка', 'ниже', 'добавить', 'друг']


0